In [1]:
import itertools
import operator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances

In [2]:
tb_df = pd.read_excel('tb_des_df.xlsx')

In [3]:
des_df = tb_df['description']

In [4]:
tb_users = pd.read_excel('tb_users_0410.xlsx')

In [5]:
des_df = pd.concat([tb_df['id'], tb_df.title, tb_df.category, des_df], axis=1)

In [6]:
des_df.head()

,id,title,category,description
0,12,사슴을 타고 온 아이,동화,\n 저를 모르시는 모든 분들께 인사드릴까요. 김가경이라고 합니다.영화와 ...
1,16,LETStudent,공간디자인,"\n안녕하세요 국민대학교 시각디자인학과 4학년에 재학중인 김문규, 민준기 입니다. ..."
2,21,Starting Up,다큐멘터리,\n 안녕하십니까 국민대학교 시각디자인과 졸업반에 정원도입니다. 이번에 제...
3,22,전방위이종문화리뷰지 나불나불,저널리즘,\nnabulnabul.com / twitter @nabulnabul\n1. 반가워...
4,23,개똥벌레: 프로그래밍 가능한 자전거 후미등,하드웨어,\n개똥벌레 소개개똥벌레는 소리로 빛의 세기를 조절하는 자전거 후미등 입니다. 자전...


In [7]:
import gensim

In [8]:
analyze = CountVectorizer(analyzer="word").build_analyzer()

In [9]:
sentences = []
for i in des_df['description']:
    words = analyze(i)
    sentences.append(words)

In [10]:
text_all=[]
for i in np.arange(len(sentences)):
    text = gensim.models.doc2vec.LabeledSentence(words = sentences[i], tags = ['doc_{number}'.format(number=i)])
    text_all.append(text)

In [11]:
import konlpy
from konlpy.tag import Kkma

In [12]:
kkma = Kkma()

In [13]:
komoran = konlpy.tag.Komoran()

In [14]:
feature_pos = ('NNP', 'NNG', 'NNB', 'NP', 'VV', 'VA', 'MAG')

In [15]:
len(des_df)

2802

In [16]:
projects_id = []
for i in np.arange(len(des_df)):
    if len(des_df['description'][i]) <=2:
        project_id = des_df.ix[i]['id']
        projects_id.append(project_id)
        print(des_df.ix[i]['id'])

123
253
371
395
466
468
471
511
528
587
597
623
628
633
647
704
705
740
757
763
769
770
804
807
816
828
868
873
882
887
892
933
966
1002
1006
1067
1134
1333
1334
1354
1409
3462
3522


In [25]:
projects_id = []
for i in np.arange(len(des_df)):
    if len(des_df['description'][i]) <=3:
        project_id = i
        projects_id.append(project_id)
        print(des_df.ix[i]['id'])

557
631
714


In [26]:
projects_id

[338, 384, 438]

In [27]:
des_df.drop(projects_id, inplace=True)

In [28]:
des_df.index = np.arange(len(des_df))

In [22]:
#kkma (konlpy)
import time
start = time.clock()

error_des_len = []
tags = []
kon_words = []
kon_words_tags = []
for i in des_df['description'][0:]:
    print(len(i))
    try:
        kon_word = kkma.pos(i)
        kon_words.append(kon_word)
    except:
        error_des_len.append(len(i))
        continue
        
#token선택
des_token =[]
for i in range(0, len(kon_words)):
    tokens = []
    for j in range(0, len(kon_words[i])):
        if kon_words[i][j][1] in feature_pos:
            tokens.append(kon_words[i][j][0])
    des_token.append(tokens)
    
end = time.clock()        
print ("%.2gs" % (end-start))

966
1760
1508
2990
1126
2536
875
2585
1652
175
2425
1984
1207
83
1432
2808
2068
2247
1963
2039
1583
1227
1368
2125
3870
1350
759
2327
2870
2484
2488
1684
639
5626
1267
731
2966
4221
1709
2402
1236
915
281
1651
2126
1312
1918
2443
3843
4597
955
2306
1984
6120
1900
1471
1820
3012
265
1565
1472
326
78
968
1277
2923
568
2246
3053
429
741
1524
1286
1588
484
1358
841
1961
3619
1824
3429
2686
389
435
279
2004
395
125
1642
1905
2567
2335
1983
2906
3699
66
2360
403
952
4464
789
7257
326
2894
2051
2218
2246
2377
5808
4132
2065
3738
3069
1271
607
3684
1347
1094
748
2285
6054
377
1670
4978
1842
972
3990
1124
5539
2047
1482
1660
1326
4908
2151
2615
248
3083
4544
2481
3553
5666
1344
5027
1545
4027
8783
2601
979
816
1927
1119
2880
2518
3132
4971
1075
889
2737
2095
2685
1969
1692
1714
2097
2250
3745
2036
1672
457
2636
1255
1458
1061
2152
6367
2171
545
1374
1683
924
290
2547
3034
1388
1716
52
2448
3304
1404
1471
1927
2220
1703
1591
1376
318
1284
1834
1433
1176
2216
729
963
612
19
3406
163
1155
485
111


In [23]:
error_des_len

[3, 6, 4]

In [29]:
len(des_token)

2755

In [30]:
len(des_df)

2755

In [32]:
# doc2vec에 넣을 수 있는 형태로 변환 (words=[], tags=[])
import time
start = time.clock()

kon_token_tags=[]
for i in np.arange(len(kon_words)):
    kon_token_tag = gensim.models.doc2vec.LabeledSentence(words = des_token[i], tags = [tb_df['id'][i]])
    kon_token_tags.append(kon_token_tag)

end = time.clock()        
print ("%.2gs" % (end-start))

0.28s


In [36]:
len(kon_token_tags[1000][0])

548

In [37]:
#modeling
import time
start = time.clock()

model_word = gensim.models.Doc2Vec(kon_token_tags, size = 200)

end = time.clock()
print ("%.2gs" % (end-start))

34s


In [ ]:
#komoran (gensim)
import time
start = time.clock()

kon_words = []
for i in des_df['description']:
    try:
        kon_word = komoran.pos(i)
        kon_words.append(kon_word)
    except:
        continue
        
end = time.clock()        
print ("%.2gs" % (end-start))

In [ ]:
kon_words = []
for i in des_df['description']:
    try:
        kon_word = kkma.nouns(i)
        kon_words.append(kon_word)
    except:
        continue

In [ ]:
kon_word_tags=[]
for i in np.arange(len(kon_words)):
    kon_word_tag = gensim.models.doc2vec.LabeledSentence(words = kon_words[i], tags = [tb_df['id'][i]])
    kon_word_tags.append(kon_word_tag)

In [ ]:
kon_sentences = []
for i in des_df['description']:
    try:
        kon_sentence = kkma.sentences(i)
        kon_sentences.append(kon_sentence)
    except:
        continue

In [ ]:
kon_sentence_tags=[]
for i in np.arange(len(kon_sentences)):
    kon_sentence_tag = gensim.models.doc2vec.LabeledSentence(words = kon_sentences[i], tags = [tb_df['id'][i]])
    kon_sentence_tags.append(kon_sentence_tag)

In [ ]:
kon_word_tags[0]

In [ ]:
kon_sentence_tags[0]

In [ ]:
len(kon_word_tags[1000][0])

In [ ]:
kon_word_tags[1000]

* 모델 training

In [ ]:
import time
start = time.clock()

model_word = gensim.models.Doc2Vec(kon_word_tags, size = 100)

end = time.clock()
print ("%.2gs" % (end-start))

In [ ]:
import time
start = time.clock()

model_sentence = gensim.models.Doc2Vec(kon_sentence_tags, size=20)

end = time.clock()
print ("%.2gs" % (end-start))

In [ ]:
model_word.save('model_word.model')

In [ ]:
model_sentence.save('model_sentence.model')